This notebook is part of the *orix* documentation https://orix.readthedocs.io. Links to the documentation won’t work from the notebook.

# Clustering misorientations

In this tutorial we will cluster Ti crystal misorientations using data obtained from a highly deformed specimen, using EBSD, as presented in <cite data-cite="johnstone2020density">Johnstone et al. (2020)</cite>. The data can be downloaded to your local cache via the [orix.data](../reference/generated/orix.data.rst) module.

Import orix classes and various dependencies

In [ ]:
# Exchange "inline" for "notebook" (or "qt5" from pyqt) for interactive plotting
%matplotlib inline

from matplotlib.colors import to_rgb
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import label2rgb
from sklearn.cluster import DBSCAN

from orix import data, plot
from orix.quaternion import Orientation, Misorientation, Rotation
from orix.vector import Vector3d


plt.rcParams.update({"font.size": 20, "figure.figsize": (10, 10)})

## Import data

Load Ti orientations with the point group symmetry *622* (*D6*). We have to explicitly allow downloading from an external source.

In [ ]:
ori = data.ti_orientations(allow_download=True)
sym = ori.symmetry
ori

Reshape the orientation mapping data to the correct spatial dimension for the scan and select a subset of the orientations with a suitable size for this demonstration (bottom left corner)

In [ ]:
ori = ori.reshape(381, 507)
ori = ori[-100:, :200]

Plot orientation maps

In [ ]:
ckey = plot.IPFColorKeyTSL(sym)

directions = [(1, 0, 0), (0, 0, 1)]
titles = ["X", "Z"]

fig, axes = plt.subplots(ncols=2, figsize=(15, 10))
for i, ax in enumerate(axes):
    ckey.direction = Vector3d(directions[i])
    ax.imshow(ckey.orientation2color(ori))
    ax.set_title(f"IPF-{titles[i]}")
    ax.axis("off")

# Add color key
ax_ipfkey = fig.add_axes(
    [0.932, 0.37, 0.1, 0.1],  # (Left, bottom, width, height)
    projection="ipf",
    symmetry=ori.symmetry.laue,
)
ax_ipfkey.plot_ipf_color_key()
ax_ipfkey.set_title("")
fig.subplots_adjust(wspace=0.01)

Map the orientations into the Rodrigues fundamental zone (find symmetrically equivalent orientations with the smallest angle of rotation) of *622*

In [ ]:
ori = ori.map_into_symmetry_reduced_zone()

Compute misorientations $m_{AB} = g_B \cdot g_A^{-1}$ (in the horizontal direction)

In [ ]:
mori_all = Misorientation(ori[:, :-1] * ~ori[:, 1:], symmetry=(sym, sym))

Keep only misorientations with a disorientation angle higher than 7$^{\circ}$, assumed to represent grain boundaries

In [ ]:
boundary_mask = mori_all.angle > np.deg2rad(7)
mori = mori_all[boundary_mask]

Map the misorientations into the Mackenzie fundamental zone of *622-622*

In [ ]:
mori = mori.map_into_symmetry_reduced_zone()

## Compute distance matrix

In [ ]:
# Increase the chunk size for a faster but more memory intensive computation
D = mori.get_distance_matrix()

## Clustering

Apply mask to remove small misorientations associated with grain orientation spread

In [ ]:
small_mask = mori.angle < np.deg2rad(7)
D = D[~small_mask][:, ~small_mask]
mori = mori[~small_mask]

For parameter explanations of the DBSCAN algorithm (Density-Based Spatial Clustering for Applications with Noise), see the [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

In [ ]:
# Compute clusters
D = D.astype(np.float32)
dbscan = DBSCAN(eps=0.05, min_samples=10, metric="precomputed").fit(D)

unique_labels, all_cluster_sizes = np.unique(
    dbscan.labels_, return_counts=True
)
print("Labels:", unique_labels)

n_clusters = unique_labels.size - 1
print("Number of clusters:", n_clusters)

Calculate the mean misorientation associated with each cluster

In [ ]:
unique_cluster_labels = unique_labels[1:]  # Drop "no-cluster" label -1
cluster_sizes = all_cluster_sizes[1:]

rc = Rotation.from_axes_angles((0, 0, 1), 15, degrees=True)

mori_mean = []
for label in unique_cluster_labels:
    # Rotate
    mori_i = rc * mori[dbscan.labels_ == label]

    # Map into the fundamental zone
    mori_i.symmetry = (sym, sym)
    mori_i = mori_i.map_into_symmetry_reduced_zone()

    # Get the cluster mean
    mori_i = mori_i.mean()

    # Rotate back and add to list
    cluster_mean_local = ~rc * mori_i
    mori_mean.append(cluster_mean_local)

cluster_means = Misorientation.stack(mori_mean).flatten()

# Map into the fundamental zone
cluster_means.symmetry = (sym, sym)
cluster_means = cluster_means.map_into_symmetry_reduced_zone()
cluster_means

Inspect misorientations in the axis-angle representation

In [ ]:
cluster_means.axis

In [ ]:
np.rad2deg(cluster_means.angle)

Define reference misorientations associated with twinning orientation relationships

In [ ]:
# From Krakow et al.
twin_theory = Rotation.from_axes_angles(
    axes=[
        (1, 0, 0),  # sigma7a
        (1, 0, 0),  # sigma11a
        (2, 1, 0),  # sigma11b
        (1, 0, 0),  # sigma13a
        (2, 1, 0),  # sigma13b
    ],
    angles=[64.40, 34.96, 85.03, 76.89, 57.22],
    degrees=True,
)

Calculate difference, defined as minimum rotation angle, between measured and theoretical values.
This procedure accounts for the edges of the fundamental zone

In [ ]:
mori2 = twin_theory.outer(cluster_means)
sym_ops = sym.outer(sym).unique()
mori2_equiv = (
    sym.outer(twin_theory).outer(sym_ops).outer(cluster_means).outer(sym)
)
D2 = mori2_equiv.angle.min(axis=(0, 2, 4))

In [ ]:
np.rad2deg(D2)

We see that the first, second, third, and fourth clusters are within $4.5^{\circ}$ of $\Sigma7$a, $\Sigma13$a, $\Sigma11$a, and $\Sigma13$b, respectively.

## Visualisation

Associate colours with clusters for plotting

In [ ]:
colors = [to_rgb(f"C{i}") for i in range(cluster_means.size)]
labels_rgb = label2rgb(dbscan.labels_, colors=colors, bg_label=-1)

Inspect misorientation axes of clusters in an inverse pole figure

In [ ]:
cluster_sizes = all_cluster_sizes[1:]
cluster_sizes_scaled = 1000 * cluster_sizes / cluster_sizes.max()

fig, ax = plt.subplots(
    figsize=(5, 5), subplot_kw=dict(projection="ipf", symmetry=sym)
)
ax.scatter(
    cluster_means.axis, c=colors, s=cluster_sizes_scaled, alpha=0.5, ec="k"
)

Plot a top view of the misorientation clusters within the fundamental zone for the (*D6*, *D6*) bicrystal symmetry

In [ ]:
wireframe_kwargs = dict(
    color="black", linewidth=0.5, alpha=0.1, rcount=361, ccount=361
)
fig = mori.scatter(
    projection="axangle",
    wireframe_kwargs=wireframe_kwargs,
    c=labels_rgb.reshape(-1, 3),
    s=4,
    alpha=0.5,
    return_figure=True,
)
ax = fig.axes[0]
ax.view_init(elev=90, azim=-60)

handle_kwds = dict(marker="o", color="none", markersize=10)
handles = []
for i in range(n_clusters):
    line = Line2D(
        [0], [0], label=i + 1, markerfacecolor=colors[i], **handle_kwds
    )
    handles.append(line)
ax.legend(
    handles=handles,
    loc="upper left",
    numpoints=1,
    labelspacing=0.15,
    columnspacing=0.15,
    handletextpad=0.05,
);

Plot side view of misorientation clusters in the fundamental zone for the (*D6*, *D6*) bicrystal symmetry

In [ ]:
fig2 = mori.scatter(
    return_figure=True,
    wireframe_kwargs=wireframe_kwargs,
    c=labels_rgb.reshape(-1, 3),
    s=4,
)
ax2 = fig2.axes[0]
ax2.view_init(elev=0, azim=-60)

Generate map of boundaries colored according to cluster membership

In [ ]:
mapping = np.ones(mori_all.shape + (3,))
new_mask = (
    np.where(boundary_mask)[0][~small_mask],
    np.where(boundary_mask)[1][~small_mask],
)
mapping[new_mask] = labels_rgb

Plot map of boundaries colored according to cluster membership

In [ ]:
fig3, ax3 = plt.subplots(figsize=(15, 10))
ax3.imshow(mapping)
ax3.set(xticks=[], yticks=[]);